# Video classification with Tensorflow 2-GPU and rolling prediction averaging

Imports Libraries

In [14]:
from tensorflow.keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2

Argument Parser

In [22]:
#construct arguments and parse the arguments
ap=argparse.ArgumentParser()

ap.add_argument("--model",required=True,help="path to serialize trained model")
ap.add_argument("--label_bin",required=True,help="path to serialize the label_bin")
ap.add_argument("--input",required=True,help="path to input video")
ap.add_argument("--output",required=True,help="path to output save video")

args=vars(ap.parse_args(["--model",r"C:\Users\SRKT\Desktop\sports_activity_dataset\model\model.h5",
                        "--label_bin",r"C:\Users\SRKT\Desktop\sports_activity_dataset\label\lb_bin",
                        "--input",r"C:\Users\SRKT\Desktop\sports_activity_dataset\video\4456-16_700273.avi",
                        "--output",r"C:\Users\SRKT\Desktop\sports_activity_dataset\output\1.avi"]))

Loads Trained Models

In [16]:
#load the trained model and label binarizer
model=load_model(args["model"])
lb=pickle.loads(open(args["label_bin"],"rb").read())
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=128)


Running Video Capture & Classification

In [23]:
#initialize the video stream 
vs=cv2.VideoCapture(args["input"])
(w,h)=(None,None)
writer=None

#loop over frames 
while True:
    (rest,frame)=vs.read()
    if not rest:
        break
    
    if w is None or h is None:
        (h,w)=frame.shape[:2]
    
    output=frame.copy()
    
    #convert frame from BGR TO RGB
    frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    frame=cv2.resize(frame,(224,224)).astype("float32")
    frame-=mean
    
    #make predictions
    preds=model.predict(np.expand_dims(frame,axis=0))[0] #expend dimensions as tensorflow accepty 3 dimensions
    Q.append(preds)
    
    #perform predcitions averaging over the current history of the previous predictions
    results=np.array(Q).mean(axis=0)
    i=np.argmax(results)
    label=lb.classes_[i]
    
    text="activity:{}".format(label)
    cv2.putText(output,text,(35,50),cv2.FONT_HERSHEY_SIMPLEX,1.25,(0,255,0),5)
    
    #check if video writer is none
    if writer is None:
        #initialize our video writer
        fourcc=cv2.VideoWriter_fourcc(*"MJPG")
        writer=cv2.VideoWriter(args["output"],fourcc,30,(w,h),True)
    writer.write(output)
    
    cv2.imshow("output",output)
    key=cv2.waitKey(1) &0xFF
    
    if key==ord("q"):
        break
        

cv2.destroyAllWindows()
    
    